In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print("adding following folder to path: ",module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

adding following folder to path:  /home/enis/projects/nna/src


In [2]:
import numpy as np 
from numpy import linspace
import pandas as pd


import datetime
from scipy import stats
import time
import copy
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import precision_recall_fscore_support


import csv 
from pathlib import Path
from os import listdir
import pickle

from tabulate import tabulate


from datetime import datetime,timedelta

import itertools


import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
from matplotlib.ticker import NullFormatter
from matplotlib.colors import ListedColormap
from matplotlib import cm

from pytz import timezone




In [3]:
from pre_process_func import read_queue
from fileUtils import read_file_properties
from labeling_utils import load_labels
from visUtils import get_cycle,createTimeIndex,file2TableDict,reverseTableDict

In [4]:
# INPUT_DIR_PARENT = "/home/data/nna/stinchcomb/NUI_DATA/"


In [20]:
def createFigure(months,monthsTime):
    plt.rcParams["axes.prop_cycle"] = get_cycle("tab10",N=8)
    

    fig, ax = plt.subplots(figsize=(80,len(months)*9),nrows=len(months),sharex=True, sharey=True,gridspec_kw={'hspace': 0})
    ax = np.array(ax).reshape(-1) # subplot returns single element for single row

    markers = itertools.cycle((',', '+', '.', 'o', '*')) 

    weather_colors=["firebrick","darkorange","green","seagreen","lightpink"]

    for monthi,month in enumerate(months):
        # for col in df_freq.columns:
        for i,(col,(lat,long)) in enumerate(cord_list):
#             if col in weather_cols:
#                 index=weather_cols.index(col)
#                 ax[monthi].plot_date(month.index.to_pydatetime(), month[col],linestyle="-",marker=" ",color=weather_colors[index])
#             else:
            ax[monthi].plot_date(month.index.to_pydatetime(), month[col],linestyle="-",marker=" ")


    ax[0].legend( labels=[id2name.get(x[0],x[0][1:]) for x in cord_list],loc='upper left', 
                borderpad=0.2, labelspacing=0.2, fontsize=28, 
                frameon=True) # frameon=False to remove frame.




    ax[-1].set_xlabel('Day Number', fontsize=32)

#     uniqueYears=pd.unique([month.year for month in monthsTime])
#     uniqueYears.size
    for i,an_ax in enumerate(ax):
        
        an_ax.set_ylabel('{}'.format(monthsTime[i].strftime("%Y-%B")),fontsize=48) #, fontweight='black')

        locator=dates.DayLocator()
        an_ax.xaxis.set_minor_locator(locator)

        an_ax.xaxis.set_minor_formatter(dates.DateFormatter('%d\n'))

        an_ax.xaxis.grid(True, which="minor")
        an_ax.xaxis.grid(True, which="major")


        an_ax.xaxis.set_major_locator(dates.AutoDateLocator())
        an_ax.xaxis.set_major_formatter(dates.DateFormatter('%d\n'))
        

        an_ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.0f}'))
        an_ax.yaxis.grid()
        an_ax.tick_params(labelsize=22,which="minor")
        an_ax.tick_params(labelsize=25,which="major")

    plt.tight_layout()
    plt.margins(x=0)
    plt.subplots_adjust(top=0.90)


    fig.suptitle('Site {}, Normalized Bi-hourly Frequency [%]'.format(selected_area),fontsize=48)
#     plt.show()

    figDir= Path(visFilePath) / ("Freq-"+freq) / regionName 
    figDir.mkdir(parents=True,exist_ok=True)
    figPath= figDir / ("_".join([selected_area,str(year)]) +'.'+"png")
    
    fig.savefig(figPath)
#     fig.savefig("test" +'.png')
    plt.close(fig)


In [15]:
from fileUtils import standardPathStyle,list_files
from visUtils import loadResults

def file2TableDict(selected_areas,model_tag_names,globalindex,globalcolumns,
                    file_properties_df,freq,dataFreq="10S",dataThreshold=0.5,
                    channel=1,gathered_results_perTag=None,
                    result_path=None,fileNameAddon="",prob2binaryFlag=True):
    # using gathered_results_perTag dictionary or  result_path to create
    # a pandas dataframe for visualizations

    # dataFreq is sampling frequency of the data,
    #most of the time we have predictions for each 10 second

    df_dict={key: None for (key) in selected_areas}
    no_result_paths=[]

    #we need to load it from files
    if gathered_results_perTag==None and (result_path==None):
        print("ERROR: gathered_results_perTag or (result_path and subDirectoryAddon )should be defined")
        return (None,None)


    for i,area in enumerate(selected_areas):
        df_sums = pd.DataFrame(index=globalindex, columns=globalcolumns).fillna(0)
        df_count = pd.DataFrame(index=globalindex, columns=globalcolumns).fillna(0)

        for modelTagName in model_tag_names:
            dfRawList=[]
    #         for afile in selected_areas_dict[area]:
            area_filtered=file_properties_df[file_properties_df.site_id==area]
            for afile,row in area_filtered.iterrows():
        #         data=gathered_results[afile][0]
                afile=Path(afile)
                # we either load data from multiple files or from single one
                if gathered_results_perTag==None:
                    # TODO, make _FCmodel variable
                    checkFolder=standardPathStyle(result_path,row,subDirectoryAddon=modelTagName
                                        ,fileNameAddon=fileNameAddon)
                    allSegments = list_files(str(checkFolder)+"/")
                    if not allSegments:
                        data=np.empty(0)
                    else:
                        data=loadResults(allSegments,prob2binaryFlag=prob2binaryFlag,
                                        threshold=dataThreshold,channel=channel)
                        # gathered_results[file]=result[:]
                else:
                    data=gathered_results_perTag[modelTagName].get(afile,np.empty(0))[:]
                    if data.size!=0 and prob2binaryFlag==True:
                        data=prob2binary(data,threshold=0.5,channel=channel)

                if data.size==0:
                    no_result_paths.append(afile)
                    continue

                start=file_properties_df.loc[afile]["timestamp"]
                end =start+timedelta(seconds=(10*(len(data)-1)))
                index = pd.date_range(start,end, freq=dataFreq)
                df_afile=pd.DataFrame(data,index=index,columns=[modelTagName])


                dfRawList.append(df_afile)
            if dfRawList:
                dfRaw=pd.concat(dfRawList)
                dfRaw=dfRaw.sort_index()
                
                df_afile_grouped = dfRaw.groupby([pd.Grouper(freq=freq)])
                counts=df_afile_grouped.count()
                sums=df_afile_grouped.sum()

                df_count=df_count.add(counts, fill_value=0) #df_count.update(counts)
                df_sums=df_sums.add(sums, fill_value=0) #df_sums.update(sums)

            
        df_dict[area]=(df_count.copy(),df_sums.copy())

    return df_dict,no_result_paths


In [5]:
linestyle_str = [
     ('solid', 'solid'),      # Same as (0, ()) or '-'
     ('dotted', 'dotted'),    # Same as (0, (1, 1)) or '.'
     ('dashed', 'dashed'),    # Same as '--'
     ('dashdot', 'dashdot'),  # Same as '-.
     ('densely dotted',        (0, (1, 1))),
     ('densely dashed',        (0, (5, 1))),
    ('densely dashdotted',    (0, (3, 1, 1, 1))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))]


In [6]:
dataFlder="/home/enis/projects/nna/data/"
resultsFlder="/home/enis/projects/nna/results/"


In [7]:
id2name={}
id2name["_CABLE"]="Cable"
id2name["_RUNNINGWATER"]="Running Water"
id2name["_INSECT"]="Insect"
id2name["_RAIN"]="Rain"
id2name["_WATERBIRD"]="Water Bird"
id2name["_WIND"]="Wind"
id2name["_SONGBIRD"]="Songbird"
id2name["_AIRCRAFT"]="Aircraft"



In [8]:
file_properties_df=pd.read_pickle("../../data/stinchcomb_dataV1.pkl")
# file_properties_df=pd.read_pickle("../../data/realdata_v2No_stinchcomb.pkl")

#important to keep them in order
file_properties_df.sort_values(by=['timestamp'],inplace=True)

# delete older than 2016
fromtime=datetime(2016, 1, 1, 0)
file_properties_df=file_properties_df[file_properties_df.timestamp>=fromtime]
all_areas=sorted(pd.unique(file_properties_df.site_id.values))

In [21]:
# file_properties_df[file_properties_df.site_id=="15-FishCreek1"]

In [9]:
# PARAMS
# FREQS to reduce results 
freq="2H"


# possible places to pick
# sorted(pd.unique(file_properties_df.site_id.values))
# areas to be visualized



In [10]:

# globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)

# selected_tag_name="_SONGBIRD"
model_tag_names=["CABLE","RUNNINGWATER","INSECT", "RAIN", "WATERBIRD", "WIND", "SONGBIRD", "AIRCRAFT"]

model_tag_names=["_"+i for i in model_tag_names]
selected_tag_name=model_tag_names[:]

weather_cols=[]
globalcolumns=model_tag_names#selected_areas+weather_cols

visFilePath="/home/enis/projects/nna/results/vis/AreaBased/"

In [11]:
# print(" Duration of selected data period:",(all_end-all_start).days,"days")
# print(" Starts: {} \n Ends:   {}".format(all_start,all_end))

In [23]:
%%time
result_path="/scratch/enis/data/nna/real/"

for selected_area in all_areas[:]:
    print(selected_area,all_areas.index(selected_area))
    selected_areas=[selected_area,]
    
    regionName=file_properties_df[file_properties_df.site_id==selected_area][:1].region[0]

    globalindex,all_start,all_end=createTimeIndex(selected_areas,file_properties_df,freq)

    df_dict,no_result_paths = file2TableDict(selected_areas,model_tag_names,globalindex,
                                             globalcolumns,file_properties_df,freq,dataFreq="10S",
                                             result_path=result_path,prob2binaryFlag=False)

    print("{} number of files do not have results".format(len(no_result_paths)))

    # df_dict_reverse=reverseTableDict(selected_areas,df_dict,model_tag_names)
    df_count,df_sums=df_dict[selected_area]

    df_freq=df_sums/df_count
    # del df_freq['UMIAT']

    df_freq=df_freq*100
    df_freq=df_freq.dropna(how='all')
    # df_freq=pd.concat([df_freq, normal_weather], axis=1, join='inner')


    # cord_list=sorted(list(cord.items()),key=lambda x: x[1][0],reverse=True)

    # cord_list=list(filter(lambda x: x[0] in df_freq.columns ,cord_list))

    cord_list=[(i,(0,0)) for i in df_freq.columns]
    
    monthsTime=pd.unique(df_freq.index.strftime("%Y-%m-01"))
    monthsTime=[pd.Timestamp(i) for i in monthsTime]
    
#     monthsTime=pd.date_range("{}-{}".format(all_start.year,all_start.month),"{}-{}".format(all_end.year,all_end.month), freq='MS',)
    monthsTimeStr=["{}-{}".format(month.year,month.month) for month in monthsTime]
    # months=[df_freq.loc['2019-05':'2019-05'],df_freq.loc['2019-06':'2019-06']]
    months=[df_freq.loc[month:month] for month in monthsTimeStr]
#     months=[month for month in months if month.size>0]
    # months2=[df_freq.loc['2016-06-01':'2016-06-30'],df_freq.loc['2016-07-01':'2016-07-31'],df_freq.loc['2016-08-01':'2016-08-31']]
#     break
    # make them all same month for aligning 
    for i,month in enumerate(months):
        months[i]=month.rename(index=lambda x: x.replace(month=7,year=2019))

    uniqueYears=np.unique([month.year for month in monthsTime])
    for year in uniqueYears:
        monthsInAYear=[months[i] for i,month in enumerate(monthsTime) if month.year==year]
        monthsTimeInAYear=[monthsTime[i] for i,month in enumerate(monthsTime) if month.year==year]
        createFigure(monthsInAYear,monthsTimeInAYear)



01-Itkillik 0
8 number of files do not have results
02-Colville2 1
8 number of files do not have results
03-OceanPt 2
4 number of files do not have results
04-Colville4 3
0 number of files do not have results
05-Colville5 4
0 number of files do not have results
06-Umiruk 5
0 number of files do not have results
07-IceRd 6
4 number of files do not have results
08-CD3 7
0 number of files do not have results
09-USGS 8
16 number of files do not have results
10-Nigliq1 9
0 number of files do not have results
11-Nigliq2 10
0 number of files do not have results
12-Anaktuvuk 11
0 number of files do not have results
13-Shorty 12
0 number of files do not have results
14-Rocky 13
0 number of files do not have results
15-FishCreek1 14
0 number of files do not have results
17-FishCreek3 15
0 number of files do not have results
18-FishCreek4 16
0 number of files do not have results
19-Itkillik2 17
0 number of files do not have results
CPU times: user 1min 41s, sys: 19.7 s, total: 2min 1s
Wall time: 3

In [17]:
!conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "Stinchcomb"

True

In [80]:
pd.unique(list(zip(df_freq.index.strftime("%Y-%m"),df_freq.index)))

array([('2018-05', Timestamp('2018-05-08 20:00:00')),
       ('2018-05', Timestamp('2018-05-09 00:00:00')),
       ('2018-05', Timestamp('2018-05-09 02:00:00')), ...,
       ('2019-08', Timestamp('2019-08-27 04:00:00')),
       ('2019-08', Timestamp('2019-08-27 06:00:00')),
       ('2019-08', Timestamp('2019-08-27 08:00:00'))], dtype=object)

In [81]:
uniquMonth=pd.unique(df_freq.index.strftime("%Y-%m-01"))
uniquMonth=[pd.Timestamp(i) for i in uniquMonth]
# monthsTime=[month for month in monthsTime if month.strftime("%Y-%m") in uniquMonth]


[Timestamp('2018-05-01 00:00:00'),
 Timestamp('2018-06-01 00:00:00'),
 Timestamp('2018-07-01 00:00:00'),
 Timestamp('2018-08-01 00:00:00'),
 Timestamp('2019-05-01 00:00:00'),
 Timestamp('2019-06-01 00:00:00'),
 Timestamp('2019-07-01 00:00:00'),
 Timestamp('2019-08-01 00:00:00')]

(12, Int64Index([5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], dtype='int64'))

In [36]:
monthsTime=pd.date_range("{}-{}".format(all_start.year,all_start.month),"{}-{}".format(all_end.year,all_end.month), freq='MS',)
# monthsTimeStr=["{}-{}".format(month.year,month.month) for month in monthsTime]
# months=[df_freq.loc[month:month] for month in monthsTimeStr]
# months=[month for month in months if month.size>0]

0

'June'